# Road network availability

---

### Objective & method

In this notebook, we analyze road data availability by measuring the actual length of the road network--instead of just bytes of information. In the OSM database, roads are modelled by "way" objects (*i.e.* polylines) with a non-null value for the `highway` property, that describes the type of road (*e.g.* primary, secondary, residential, track, pedestrian, etc.).

---

In [43]:
import os
import pytz
from datetime import datetime
import subprocess
import json

import osmium
import numpy as np
import pandas as pd
import geopandas as gpd
from fiona import crs
import geojson
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
INPUT_DIR = os.path.abspath(os.path.join('..', 'data', 'input'))
OUTPUT_DIR = os.path.abspath(os.path.join('..', 'data', 'output'))
OSM_DATAFILE = os.path.join(INPUT_DIR, 'africa-internal-190515.osh.pbf')

In [3]:
africa = gpd.read_file('../data/input/africa.geojson')
africa.set_index('ADMIN', drop=True, inplace=True)
africa.drop(columns='ISO_A3', inplace=True)
# Output data from previous notebook
densities = pd.read_csv('../data/output/osmdb_densities.csv', index_col=0)

* https://osm-internal.download.geofabrik.de/africa-internal.osh.pbf

## Geographic extracts

* https://oslandia.com/en/2017/07/03/openstreetmap-data-analysis-how-to-parse-the-data-with-python/
* https://osmcode.org/osmium-tool/manual.html#creating-geographic-extracts
* https://github.com/jazzband/python-geojson/issues/84

The following analysis focuses on the road network, so we first filter input data based on the `highway` OSM tag in order to reduce the size of the dataset.

In [14]:
if not os.path.isfile('../data/input/africa-highways.osh.pbf'):

    subprocess.run([
        'osmium', 'tags-filter', '../data/input/africa-190605.osh.pbf', 'w/highway',
        '-o', '../data/input/africa-highways.osh.pbf'
    ])

Then we perform a geographical extract for each country using the `osmium extract` subcommand.

In [18]:
# Create geographical extracts for each country
outputdir = '../data/input/osm'
os.makedirs(outputdir, exist_ok=True)
for country in africa.index:

    # Avoid if file already exists
    if os.path.isfile(os.path.join(outputdir, cid + '.osh.pbf')):
        continue

    # Generate ID from country name
    cid = country.replace(' ', '_')

    # Dump geometry into a GeoJSON file
    geom = africa.at[country, 'geometry'].__geo_interface__
    with open('/tmp/geom.geojson', 'w') as f:
        geojson.dump(geojson.Feature(geometry=geom), f)

    # Extract from main OSM datafile with osmium
    subprocess.run([
        'osmium', 'extract', '--with-history',
        '-p', '/tmp/geom.geojson',
        '../data/input/africa-highways.osh.pbf',
        '-o', os.path.join(outputdir, cid + '.osh.pbf')
    ])
    
    os.remove('/tmp/geom.geojson')

## Road network analysis

* https://wiki.openstreetmap.org/wiki/Highways

In [91]:
TAGS = [
    'highway',
    'maxspeed',
    'oneway',
    'width',
    'surface',
    'incline',
    'foot',
    'bicycle',
    'access',
    'bus',
    'tracktype',
    'lit',
    'lanes',
    'sidewalk',
    'crossing'
]

In [93]:
class RoadsHandler(osm.SimpleHandler):
    
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.data = []
    
    def way(self, w):
        if 'highway' in w.tags:
            try:
                length = osm.geom.haversine_distance(w.nodes)
            except osmium.InvalidLocationError:
                # Way incomplete (ignore)
                return
            self.data.append([
                w.id,
                w.version,
                pd.Timestamp(w.timestamp),
                w.tags['highway'],
                length
            ])

In [94]:
handler = RoadsHandler()
handler.apply_file('../data/input/osm/Mauritania.osh.pbf', locations=True)

In [101]:
columns = ['osmid', 'version', 'timestamp', 'highway', 'length']
roads = pd.DataFrame(handler.data, columns=columns)

In [98]:
def snapshot(roads, date):
    """Snapshot of the road network at a specific date."""
    data = roads.query('timestamp <= @date').groupby('osmid').version.max()
    data = data.reset_index()
    return pd.merge(data, roads, on=['osmid', 'version'])

In [112]:
def summarize(roads, date):
    """Total road length per type at a specific date."""
    roads_snapshot = snapshot(roads, date)
    lengths = roads_snapshot.groupby('highway').length.sum().round()
    return lengths / 1000

In [132]:
def history(roads, start_date, end_date, freq='3M'):
    """Evolution of road length for each road type."""
    daterange = pd.date_range(start_date, end_date, freq=freq, tz=pytz.UTC)
    summaries = [summarize(roads, date) for date in daterange]
    evolution = pd.DataFrame(summaries).T
    evolution.columns = [d.strftime('%Y-%m') for d in daterange]
    return evolution

In [135]:
start = datetime(2010, 1, 1, tzinfo=pytz.UTC)
end = datetime(2019, 5, 1, tzinfo=pytz.UTC)
evolution = history(roads, start, end)

### Semantic information